In [366]:
import pandas as pd
import config
import json
import sys
import numpy as np

In [367]:
def process_ips_base_file():
    df_base_file = pd.read_csv('C:/Users/pv/OneDrive - Kmart Australia Limited/KHome/My Offline Files/Desktop/Detail/IPS_all.csv')
    df_base_file = pd.pivot_table(df_base_file, values='LINE_VALUE', index=['PRODUCT_GENERATED_IDENTIFIER','LOCATION_GENERATED_IDENTIFIER','PRODUCT_SOURCE_IDENTIFIER','LOCATION_SOURCE_IDENTIFIER','LOCATION_NAME','STATE','PRODUCT_DESCRIPTION','DEPARTMENT_DESCRIPTION','DATE_'], columns='LINE_ITEM', fill_value=0).reset_index()
    df_base_file = df_base_file.rename(columns={"MANIFEST": "IPS_MANIFEST", "SALES": "IPS_SALES", "SHRINK": "IPS_SHRINK" ,"ITC": "IPS_ITC", "PO": "IPS_PO", "RLO": "IPS_RLO"})
    return df_base_file

In [368]:
def process_mr_base_file():
    df_mr_base_file = pd.read_csv('C:/Users/pv/OneDrive - Kmart Australia Limited/KHome/My Offline Files/Desktop/Detail/MR_all.csv')
    df_mr_base_file = pd.pivot_table(df_mr_base_file, values='LINE_VALUE', index=['PRODUCT_GENERATED_IDENTIFIER','LOCATION_GENERATED_IDENTIFIER','PRODUCT_SOURCE_IDENTIFIER','LOCATION_SOURCE_IDENTIFIER','LOCATION_NAME','STATE','PRODUCT_DESCRIPTION','DEPARTMENT_DESCRIPTION','DATE_'], columns='LINE_ITEM', fill_value=0).reset_index()
    df_mr_base_file = df_mr_base_file.rename(columns={"MANIFEST": "MR_MANIFEST", "SALES": "MR_SALES", "SHRINK": "MR_SHRINK" ,"ITC": "MR_ITC", "PO": "MR_PO", "RLO": "MR_RLO"})
    return df_mr_base_file

In [369]:
def generate_rank(df_base_file, df_mr_base_file):
    both_df = pd.merge(df_base_file, df_mr_base_file, how='inner', on=['PRODUCT_GENERATED_IDENTIFIER', 'LOCATION_GENERATED_IDENTIFIER'] ).drop_duplicates()
    rnk_df = both_df[['PRODUCT_GENERATED_IDENTIFIER','LOCATION_GENERATED_IDENTIFIER']]
    rnk_df["Rank"] = both_df[['PRODUCT_GENERATED_IDENTIFIER','LOCATION_GENERATED_IDENTIFIER']].apply(tuple,axis=1).rank(method='dense',ascending=False).astype(int)
    rnk_df = rnk_df.sort_values("Rank").drop_duplicates()
    return rnk_df

In [370]:
df_base_file = process_ips_base_file()
df_mr_base_file = process_mr_base_file()
rnk_df = generate_rank(df_base_file, df_mr_base_file)

df_base_file = pd.merge(df_base_file, rnk_df, how='inner', on=['PRODUCT_GENERATED_IDENTIFIER', 'LOCATION_GENERATED_IDENTIFIER'] ).drop_duplicates()
df_mr_base_file = pd.merge(df_mr_base_file, rnk_df, how='inner', on=['PRODUCT_GENERATED_IDENTIFIER', 'LOCATION_GENERATED_IDENTIFIER'] ).drop_duplicates()
res_ = pd.merge(df_base_file, df_mr_base_file, how='outer', on=['PRODUCT_GENERATED_IDENTIFIER', 'LOCATION_GENERATED_IDENTIFIER', 'PRODUCT_SOURCE_IDENTIFIER', 'LOCATION_SOURCE_IDENTIFIER','DATE_','LOCATION_NAME','STATE','PRODUCT_DESCRIPTION','DEPARTMENT_DESCRIPTION','Rank'])

SOH_STARTING = 0
detail_df = pd.DataFrame([])


C:\Users\pv\AppData\Local\Temp\ipykernel_91880\2580220989.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rnk_df["Rank"] = both_df[['PRODUCT_GENERATED_IDENTIFIER','LOCATION_GENERATED_IDENTIFIER']].apply(tuple,axis=1).rank(method='dense',ascending=False).astype(int)


In [371]:
from tqdm import tqdm

for i in tqdm(range(1, max(df_base_file['Rank'])+1)):
        res = res_[res_.Rank==i]
        #Calculate IPS and MR Cumulative_Sales
        res = res.sort_values(by = ['DATE_'])
        res = res.fillna(0)
        res['IPS_Cumulative_Sales'] = res['IPS_SALES'].cumsum()
        res['MR_Cumulative_Sales'] = res['MR_SALES'].cumsum()
        res = res.fillna(0)
        
        res['IPS_CALC'] = res['IPS_MANIFEST'] + res['IPS_SALES'] + res['IPS_ITC'] + res['IPS_RLO'] + res['IPS_SHRINK'] #+ res['IPS_PO']
        res['MR_CALC'] = res['MR_MANIFEST'] + res['MR_SALES'] + res['MR_ITC'] + res['MR_RLO'] + res['MR_SHRINK'] #+ res['MR_PO']
        res['IPS_Dervied_SOH'] = res['IPS_CALC'].cumsum()
        res['MR_Dervied_SOH'] = res['MR_CALC'].cumsum()

        res = res.fillna(0)
        res = res.drop_duplicates()
        
        if (((i % 100) == 0) or (i == max(df_base_file['Rank']))):
                detail_df = pd.concat([detail_df,res])
                detail_df = detail_df.drop(['IPS_CALC','MR_CALC'], axis=1)
                detail_df.to_csv("C:/Users/pv/OneDrive - Kmart Australia Limited/KHome/My Offline Files/Desktop/All_Detail/" + str(i) +".csv", index=False)
                detail_df = pd.DataFrame([])
        else: 
                detail_df = pd.concat([detail_df,res])

100%|██████████| 75254/75254 [1:26:20<00:00, 14.53it/s]      


In [380]:
import pandas as pd
import os
import glob
path = os.getcwd()
csv_files = glob.glob(os.path.join("C:/Users/pv/OneDrive - Kmart Australia Limited/KHome/My Offline Files/Desktop/All_Detail/", "*.csv"))
  
  
# loop over the list of csv files
detail_df = pd.DataFrame([])
for f in csv_files:
    df = pd.read_csv(f)
    detail_df = pd.concat([detail_df,df])

detail_df = detail_df.drop('Rank', axis=1)
      
      

In [382]:
detail_df.to_csv("C:/Users/pv/OneDrive - Kmart Australia Limited/KHome/My Offline Files/Desktop/Detail/PAPER_SHRINK_DETAILED_ALL.csv", index=False)
 